In [2]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn_pandas import DataFrameMapper, cross_val_score
import sklearn.preprocessing, sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
df = pd.read_csv('india_surroundings_1mar2016 _28_feb_2017.csv')

In [4]:
df.head()

,Year,Month,Day,Time UTC,Mag,Lat,Lon,Depth km,Region,IRIS ID,Timestamp
0,2017,2,27,04:21:42,4.6,27.35,85.96,25.7,NEPAL,10004460,1488169302
1,2017,2,27,03:37:46,4.7,27.50,86.17,10.0,NEPAL,10004453,1488166666
2,2017,2,26,18:48:45,4.8,30.28,83.56,10.0,XIZANG,10004374,1488134925
3,2017,2,26,05:57:03,4.1,36.45,71.23,103.9,AFGHANISTAN-TAJIKISTAN BORD REG.,10004276,1488088623
4,2017,2,24,12:50:59,4.3,36.25,70.00,134.9,"HINDU KUSH REGION, AFGHANISTAN",10003539,1487940659


In [5]:
del df['IRIS ID'], df['Year'], df['Month'], df['Day'], df['Time UTC']

In [6]:
df.describe()

,Mag,Lat,Lon,Depth km,Timestamp
count,280.000000,280.000000,280.000000,280.000000,2.800000e+02
mean,4.494643,31.771107,79.282536,90.975357,1.471467e+09
std,0.416428,6.894564,10.009182,71.509370,9.126734e+06
min,3.600000,8.350000,68.920000,4.100000,1.456901e+09
25%,4.200000,27.950000,70.830000,25.800000,1.463153e+09
50%,4.400000,35.870000,71.925000,90.150000,1.470943e+09
75%,4.600000,36.520000,89.397500,135.375000,1.479141e+09
max,6.900000,37.760000,96.750000,298.300000,1.488169e+09


In [7]:
mapper = DataFrameMapper([
        ('Region', None),
        (['Mag'],sklearn.preprocessing.StandardScaler()),
        (['Depth km'],sklearn.preprocessing.StandardScaler()),
        (['Lat'],sklearn.preprocessing.StandardScaler()),
        (['Lon'],sklearn.preprocessing.StandardScaler()),
        (['Timestamp'], sklearn.preprocessing.StandardScaler()) 
    ], df_out=True)
df = mapper.fit_transform(df)
df[['Mag','Lat','Lon','Timestamp','Depth km']] = df[['Mag','Lat','Lon','Timestamp','Depth km']].astype(float)

/home/nikhil/programs/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/nikhil/programs/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
#axs = pd.scatter_matrix(df, figsize=(12,12), diagonal='kde')
df.corr()

,Mag,Depth km,Lat,Lon,Timestamp
Mag,1.000000,-0.156322,-0.272058,0.334685,0.071506
Depth km,-0.156322,1.000000,0.471849,-0.569523,-0.026960
Lat,-0.272058,0.471849,1.000000,-0.785022,0.055928
Lon,0.334685,-0.569523,-0.785022,1.000000,-0.047623
Timestamp,0.071506,-0.026960,0.055928,-0.047623,1.000000


In [10]:
df = pd.get_dummies(df, columns = ['Region'])

In [11]:
df.head()

,Mag,Depth km,Lat,Lon,Timestamp,Region_AFGHANISTAN,Region_AFGHANISTAN-TAJIKISTAN BORD REG.,"Region_ANDAMAN ISLANDS, INDIA","Region_ANDAMAN ISLANDS, INDIA REGION",Region_EASTERN KASHMIR,...,Region_NORTHERN INDIA,Region_NORTHWESTERN KASHMIR,Region_PAKISTAN,"Region_QINGHAI, CHINA",Region_SOUTHEASTERN AFGHANISTAN,Region_SOUTHERN INDIA,"Region_SOUTHERN XINJIANG, CHINA",Region_TAJIKISTAN,Region_WESTERN XIZANG-INDIA BORDER REG.,Region_XIZANG
0,0.253455,-0.914457,-0.642393,0.668328,1.833269,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.494023,-1.134402,-0.620598,0.689347,1.832979,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.734591,-1.134402,-0.216660,0.428119,1.829495,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.949383,0.181064,0.679850,-0.805955,1.824413,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.468248,0.615350,0.650790,-0.929063,1.808172,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pipe = sklearn.pipeline.make_pipeline(mapper, sklearn.svm.SVC())

In [49]:
X = df[['Lat','Lon']].astype(int)
#del X['Mag']

np.round(cross_val_score(pipe, X=X, y=df.Mag., scoring='r2'), 2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [45]:
np.isfinite(df.Mag.sum())

True